In [9]:
import pandas as pd
import glob
import os

# Carpeta donde tienes los CSV
carpeta = "data"   # 👈 cámbiala si tus archivos están en otra ruta

# Buscar todos los CSV en la carpeta
archivos = glob.glob(os.path.join(carpeta, "*.csv"))

df_estudiantes = []
df_profesores = []

for archivo in archivos:
    nombre = os.path.basename(archivo).replace(".csv", "")
    
    if nombre.startswith("E"):
        periodo = int(nombre[1:])
        df = pd.read_csv(archivo, encoding="latin1", sep=";")
        df["Periodo"] = periodo
        df_estudiantes.append(df)
        
    elif nombre.startswith("P"):
        periodo = int(nombre[1:])
        df = pd.read_csv(archivo, encoding="latin1", sep=";")
        df["Periodo"] = periodo
        df_profesores.append(df)

# --- Unir ---
df_estudiantes = pd.concat(df_estudiantes, ignore_index=True)
df_profesores = pd.concat(df_profesores, ignore_index=True)

# --- Limpieza ---
# 1. Dentro de cada periodo: eliminar duplicados por Persona
df_estudiantes = df_estudiantes.drop_duplicates(subset=["Persona", "Periodo"])
df_profesores = df_profesores.drop_duplicates(subset=["Persona", "Periodo"])

# 2. Eliminar de estudiantes los que también sean profesores en el mismo periodo
codigos_profesores = set(df_profesores["Persona"].unique())
mask = ~(df_estudiantes["Persona"].isin(codigos_profesores) & 
         df_estudiantes["Periodo"].isin(df_profesores["Periodo"]))
df_estudiantes = df_estudiantes[mask]

# --- Ajustar tipos (enteros como enteros, decimales se respetan) ---
def fix_types(df):
    for col in df.columns:
        if pd.api.types.is_float_dtype(df[col]):
            # si todos los valores no nulos son enteros
            if df[col].dropna().apply(float.is_integer).all():
                df[col] = df[col].astype("Int64")  # enteros con soporte de nulos
    return df

df_profesores = fix_types(df_profesores)
df_estudiantes = fix_types(df_estudiantes)

# --- Guardar archivos finales ---
df_profesores = df_profesores.sort_values(by=["Periodo", "Persona"]).reset_index(drop=True)
df_estudiantes = df_estudiantes.sort_values(by=["Periodo", "Persona"]).reset_index(drop=True)

df_profesores.to_csv("Profesores_Unificado.csv", index=False, encoding="latin1", sep=";")
df_estudiantes.to_csv("Estudiantes_Unificado.csv", index=False, encoding="latin1", sep=";")

print("✅ Archivos generados: Profesores_Unificado.csv y Estudiantes_Unificado.csv")

✅ Archivos generados: Profesores_Unificado.csv y Estudiantes_Unificado.csv
